In [1]:
#!pip install -q duckdb
#!pip install -q duckdb pyarrow

import os
import time
import subprocess
import duckdb
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from IPython.display import display

# КОНФИГУРАЦИЯ

# 1. Путь к сырым данным (Архивы)
RAW_INPUT_DIR = '/kaggle/input/kkbox-churn-prediction-challenge'

# 2. Путь к подготовленным в parquet данным
SILVER_DATASET_DIR = '/kaggle/input/kkbox-silver-parquet-kucheriavykh'

# 3. Рабочая папка
WORKING_DIR = '/kaggle/working'

In [2]:
def get_data_paths():
    """
    Возвращает словарь с путями к 4 файлам.
    Приоритет:
    1. Готовый датасет (Input) -> Мгновенно
    2. Рабочая папка (Working) -> Мгновенно (при рестарте ядра)
    3. Генерация с нуля (ETL) -> Долго (Fallback)
    """
    
    # Ожидаемые файлы
    files = {
        'train': 'train.parquet',
        'members': 'members.parquet',
        'transactions': 'transactions.parquet',
        'user_logs': 'user_logs.parquet'
    }
    
    paths = {}
    missing_files = []

    print("ПОИСК ДАННЫХ...")

    # --- СТРАТЕГИЯ 1: ИЩЕМ В ПОДКЛЮЧЕННОМ ДАТАСЕТЕ (SILVER LAYER) ---
    if os.path.exists(SILVER_DATASET_DIR):
        all_found = True
        temp_paths = {}
        for key, filename in files.items():
            full_path = f"{SILVER_DATASET_DIR}/{filename}"
            if os.path.exists(full_path):
                temp_paths[key] = full_path
            else:
                all_found = False
                break
        
        if all_found:
            print(f"УСПЕХ: Найден подключенный датасет в {SILVER_DATASET_DIR}")
            print("Этап ETL пропущен.")
            return temp_paths
    
    print("Готовый датасет не найден или неполный.")

    # --- СТРАТЕГИЯ 2: ИЩЕМ В РАБОЧЕЙ ПАПКЕ (WORKING) ---
    for key, filename in files.items():
        full_path = f"{WORKING_DIR}/{filename}"
        if os.path.exists(full_path):
            paths[key] = full_path
        else:
            missing_files.append(key)
            
    if not missing_files:
        print("УСПЕХ: Найдены файлы в рабочей директории (cache).")
        return paths

    # --- СТРАТЕГИЯ 3: ЗАПУСК ТЯЖЕЛОГО ETL (FALLBACK) ---
    print(f"КЕШ НЕ НАЙДЕН. Отсутствуют: {missing_files}")
    print("Запуск процесса генерации (займет 20-30 минут)...")
    
    # Запускаем функции генерации
    run_fallback_etl()
    
    # После генерации пути будут в working
    for key, filename in files.items():
        paths[key] = f"{WORKING_DIR}/{filename}"
        
    return paths


# КОД ГЕНЕРАЦИИ (ETL) - СПРЯТАН В ФУНКЦИЮ
def run_fallback_etl():
    """Запускается только если нет данных"""
    
    # Вспомогательные функции (Streaming и Simple)
    def process_simple(archive, csv_n, parquet_n):
        if os.path.exists(f"{WORKING_DIR}/{parquet_n}"): return
        print(f"Processing {parquet_n}...")
        os.system(f"7z x {RAW_INPUT_DIR}/{archive} -o{WORKING_DIR} -y > /dev/null")
        con = duckdb.connect()
        con.execute(f"COPY (SELECT * FROM '{WORKING_DIR}/{csv_n}') TO '{WORKING_DIR}/{parquet_n}' (FORMAT PARQUET, COMPRESSION 'ZSTD')")
        os.remove(f"{WORKING_DIR}/{csv_n}")

    def process_streaming(archive, parquet_n):
        if os.path.exists(f"{WORKING_DIR}/{parquet_n}"): return
        print(f"Streaming {parquet_n}...")
        proc = subprocess.Popen(['7z', 'x', f"{RAW_INPUT_DIR}/{archive}", '-so'], stdout=subprocess.PIPE)
        chunk_iter = pd.read_csv(proc.stdout, chunksize=1_000_000)
        writer = None
        for chunk in chunk_iter:
            table = pa.Table.from_pandas(chunk)
            if writer is None: writer = pq.ParquetWriter(f"{WORKING_DIR}/{parquet_n}", table.schema, compression='ZSTD')
            writer.write_table(table)
        if writer: writer.close()
        proc.terminate()

    # Запуск
    process_simple('train.csv.7z', 'train.csv', 'train.parquet')
    process_simple('members_v3.csv.7z', 'members_v3.csv', 'members.parquet')
    process_simple('transactions.csv.7z', 'transactions.csv', 'transactions.parquet')
    process_streaming('user_logs.csv.7z', 'user_logs.parquet')

In [3]:
# 1. Получаем пути к файлам
data_paths = get_data_paths()

# 2. Подключаемся к базе DuckDB
con = duckdb.connect()

print("Создаю SQL-связи с файлами...")

# 3. Регистрируем файлы как таблицы (Views)
for table_name, file_path in data_paths.items():
    con.execute(f"CREATE OR REPLACE VIEW {table_name} AS SELECT * FROM '{file_path}'")
    print(f"Таблица '{table_name}' готова -> смотрит на {file_path}")

print("\nГотово! Теперь можно писать SQL-запросы к таблицам: train, members, transactions, user_logs")

# 4. Просто посмотрим на сырые данные, чтобы убедиться, что всё работает
print("Пример сырых данных (таблица Train):")
df_train_sample = con.execute("SELECT * FROM train LIMIT 5").df()
display(df_train_sample)  # <--- Вот это магия

print("\nПример сырых данных (таблица Members):")
df_members_sample = con.execute("SELECT * FROM members LIMIT 5").df()
display(df_members_sample)

print("\nПример сырых данных (таблица Transactions):")
df_trans_sample = con.execute("SELECT * FROM transactions LIMIT 5").df()
display(df_trans_sample)

print("\nПример сырых данных (таблица User_logs):")
df_logs_sample = con.execute("SELECT * FROM user_logs LIMIT 5").df()
display(df_logs_sample)

ПОИСК ДАННЫХ...
УСПЕХ: Найден подключенный датасет в /kaggle/input/kkbox-silver-parquet-kucheriavykh
Этап ETL пропущен.
Создаю SQL-связи с файлами...
Таблица 'train' готова -> смотрит на /kaggle/input/kkbox-silver-parquet-kucheriavykh/train.parquet
Таблица 'members' готова -> смотрит на /kaggle/input/kkbox-silver-parquet-kucheriavykh/members.parquet
Таблица 'transactions' готова -> смотрит на /kaggle/input/kkbox-silver-parquet-kucheriavykh/transactions.parquet
Таблица 'user_logs' готова -> смотрит на /kaggle/input/kkbox-silver-parquet-kucheriavykh/user_logs.parquet

Готово! Теперь можно писать SQL-запросы к таблицам: train, members, transactions, user_logs
Пример сырых данных (таблица Train):


,msno,is_churn
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1



Пример сырых данных (таблица Members):


,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,None,11,20110911
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,None,7,20110914
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,None,11,20110915
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,None,11,20110915
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,20110915



Пример сырых данных (таблица Transactions):


,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,20150930,20160427,0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,20150930,20151128,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,20150930,20151121,0



Пример сырых данных (таблица User_logs):


,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,20150513,0,0,0,0,1,1,280.335
1,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,20150709,9,1,0,0,7,11,1658.948
2,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20150105,3,3,0,0,68,36,17364.956
3,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20150306,1,0,1,1,97,27,24667.317
4,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,20150501,3,0,0,0,38,38,9649.029
